In [4]:
import requests
import csv
import time
import pandas as pd
import numpy as np

import random

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.utils import ChromeType




from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import os.path
import time


In [5]:

URL = "https://www.nba.com/stats/events/?CFID=&CFPARAMS=&ContextMeasure=OREB&GameID=&PlayerID=0&Season=2022-23&SeasonType=Regular%20Season&TeamID=1610612749&flag=1&sct=plot&section=game"

fileName = "MISS" #This will be for later, in case we want to save
NUMVIDS = 50 #Number of video per each link

wantedFilterOn = True #Wanted Filter Bool, set to False if wanted list is empty.
#The title must have ALL substring in wanted list and NONE substring in unwanted list to get included.
wantedTitleStrList = ["MISS"] #List of substring to look for in the title.
unWantedTitleStrList = ["BLOCK"] #List of substring that will ignore in the title.

#Creating the driver
driver = webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))


wait = WebDriverWait(driver, 1)




###########################################################################################################
# AllVidsDf = pd.DataFrame()
# categoryDf = pd.read_csv('CategoryLinks.csv')
# # Putting the wanted category in the wanted string
# wantedDf = categoryDf[categoryDf['links'].str.contains(wantedLink)==True]
#
# main_url = "https://www.nba.com" + wantedDf["links"].loc[wantedDf.index[LINK_INDEX - 1]]


driver.get(URL)

time.sleep(5)
#This is to set the page to list all the video
all = None
for i in range(0,10):
    try:
        # connect
        all = wait.until(EC.presence_of_element_located((By.XPATH, "//option[@value = '-1']")))
    except:
         pass

all.click()

wait.until(EC.presence_of_element_located((By.XPATH,"//tr[@data-has-video='true']")))
AllVidElement = driver.find_elements(By.XPATH, '//tr[@data-has-video="true"]')
print(len(AllVidElement))
vidDown = 0
vidList = []
for vidButton in AllVidElement:
    driver.execute_script("arguments[0].click();", vidButton)
    videoElement = wait.until(EC.presence_of_element_located((By.XPATH, "//video")))
    vidLinks = videoElement.get_attribute("src")
    titleElement = driver.find_element(By.TAG_NAME, 'h2')
    titleString = titleElement.text
    unwantedBool = any(ele in titleString for ele in unWantedTitleStrList)
    wantedBool = (not wantedFilterOn or any(ele in titleString for ele in wantedTitleStrList))
    if not wantedBool or unwantedBool:
        continue

    #add it to our list
    vidList.append({'link' : vidLinks, 'title': titleString})
    vidDown = vidDown + 1
    print(vidDown)
    if vidDown >= NUMVIDS:
        break

vidDf = pd.DataFrame(vidList)
driver.quit()



[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:01<00:00, 4.69MB/s]


AttributeError: 'NoneType' object has no attribute 'click'

0The code below will save the links we got to an existing csv file by appending it

In [25]:
path = "Saved/"
finalFileName = path + fileName + ".csv"

try:
    test = pd.read_csv(finalFileName)
except:
    vidDf.to_csv(finalFileName, index= False,  header=True)
else:
    vidDf.to_csv(finalFileName, mode='a', index= False, header=False)

#The Code below is used to filter out the duplicates
df = pd.read_csv(finalFileName)
df = df.drop_duplicates(subset=["link"], keep="first")
print(df.shape)
df.to_csv(finalFileName, index= False, header=True)



(349, 2)
